In [1]:
import sys
import os

if sys.platform == "darwin":
    ROOT = '/Users/rohan/repositories/11785-f23-prj/faceted_visualization'
else:
    ROOT = '/apps/11785-f23-prj/faceted_visualization'

LIBRARY_HOME = os.path.join(ROOT, 'visualizer')
PROBE_HOME = os.path.join(ROOT, "probes", "logs", "flower")
LOG_PATH = os.path.join(ROOT, "logs")

sys.path.append(ROOT)
sys.path.append(LIBRARY_HOME)

In [2]:
%cd /apps/11785-f23-prj

/apps/11785-f23-prj


In [10]:
import faceted_visualization.visualizer.main
import faceted_visualization.visualizer.render
import faceted_visualization.visualizer.hook
import faceted_visualization.visualizer.image
import faceted_visualization.visualizer.helpers
import faceted_visualization.visualizer.wb
import faceted_visualization.visualizer.constants as constants
import wandb
import pprint
import clip
import os
import logging
import datetime
import torch

In [11]:
logging.basicConfig(force=True, level="INFO",
                    format="%(asctime)s [%(threadName)-12.12s] [%(levelname)-5.5s]  %(message)s")
logger = logging.getLogger()
init_stream = True
init_filehandler = True
for handler in logger.handlers:
    if isinstance(handler, logging.StreamHandler):
        init_stream = False
    if isinstance(handler, logging.FileHandler):
        init_filehandler = False
if init_stream:
    consoleHandler = logging.StreamHandler(sys.stdout)
    logger.addHandler(consoleHandler)
if init_filehandler and sys.platform != "darwin":
    fileHandler = logging.FileHandler(os.path.join(LOG_PATH, f"{datetime.datetime.now().strftime('%Y%m%d%H%M%S')}.log"))
    fileHandler.setFormatter(
        logging.Formatter(fmt="%(asctime)s [%(threadName)-12.12s] [%(levelname)-5.5s]  %(message)s"))
    logger.addHandler(fileHandler)

In [12]:
generated_random_seed = faceted_visualization.visualizer.helpers.random_seed(8)
logger.info("Seed = %s", generated_random_seed)

2023-12-04 17:58:41,314 [MainThread  ] [INFO ]  Seed = 18630595


In [13]:
parameter_dict = {
    constants.NUMBER_OF_ITERATIONS: {
        'values': [512]
#         'values': [2]
    },
    constants.MODEL: {
        'values': ['RN50x4']
    },
    constants.LINEAR_PROBE_LAYER: {
        'values': [
            'layer1_0_conv3',
            'layer1_0_relu3',
            'layer1_3_conv3',
            'layer1_3_relu3',
            'layer2_5_conv3',
            'layer2_5_relu3',
            'layer3_9_conv3',
            'layer3_9_relu3'
        ]
    },
    constants.VISUALIZATION_LAYER: {
        'values': [
            'layer4_5_conv3'
        ]
    },
    constants.OPTIMIZER: {
        'values': [
            'AdamW'
        ]
    },
    constants.LEARNING_RATE: {
        'values': [
            0.05
        ]
    },
    constants.CHANNEL: {
        'values': [
            1, 128, 512, 2048, 2560  
        ]
    },
    constants.OBJECTIVE: {
        'values': [
            'neuron',
            'channel'
        ]
    },
    constants.USE_TRANSFORMS: {
        'values': [
            True
        ]
    },
    constants.USE_STD_TRANSFORMS: {
        'values': [
            True
        ]
    },
    constants.RANDOM_SEED: {
        'values': [
            generated_random_seed
        ]
    }
}
sweep_config = {
    'method': 'grid',
    'parameters': parameter_dict

}
logger.info("Sweep Configuration = \n %s", pprint.pformat(sweep_config))

2023-12-04 17:59:48,589 [MainThread  ] [INFO ]  Sweep Configuration = 
 {'method': 'grid',
 'parameters': {'channel': {'values': [1, 128, 512, 2048, 2560]},
                'learning_rate': {'values': [0.05]},
                'linear_probe_layer': {'values': ['layer1_0_conv3',
                                                  'layer1_0_relu3',
                                                  'layer1_3_conv3',
                                                  'layer1_3_relu3',
                                                  'layer2_5_conv3',
                                                  'layer2_5_relu3',
                                                  'layer3_9_conv3',
                                                  'layer3_9_relu3']},
                'model': {'values': ['RN50x4']},
                'num_iterations': {'values': [512]},
                'objective': {'values': ['neuron', 'channel']},
                'optimizer': {'values': ['AdamW']},
                'random_se

In [14]:
if torch.cuda.is_available():
    DEVICE = "cuda:0"
else:
    DEVICE = "cpu"
logger.info("Selected Device = %s", DEVICE)

2023-12-04 17:59:58,580 [MainThread  ] [INFO ]  Selected Device = cuda:0


In [15]:
def perform_sweep(config=None):
    with wandb.init(config) as run:
        current_sweep_config = wandb.config
        logger.info('Starting sweep [ run  %s ] with config = \n%s', run.id, current_sweep_config)
        faceted_visualization.visualizer.helpers.set_seed(current_sweep_config[constants.RANDOM_SEED])
        model, clip_transforms = faceted_visualization.visualizer.main.get_model(current_sweep_config[constants.MODEL],
                                                                                 device=DEVICE)
        model_hook = faceted_visualization.visualizer.hook.register_hooks(model)
        if current_sweep_config[constants.MODEL].casefold() == 'RN50x4'.casefold():
            image_h, image_w = 288, 288
            probe_location = os.path.join(PROBE_HOME, current_sweep_config[constants.MODEL],
                                          current_sweep_config[constants.LINEAR_PROBE_LAYER], "version_4_20",
                                          "model_checkpoint.pth")
        else:
            image_h, image_w = 224, 224
            probe_location = os.path.join(PROBE_HOME,
                                          current_sweep_config[constants.MODEL] + "_" + current_sweep_config[
                                              constants.LINEAR_PROBE_LAYER], "version_1", "model_checkpoint.pth")
        params, image_f = faceted_visualization.visualizer.image.generate_img(h=image_h, w=image_w,
                                                                              decorrelate=True,
                                                                              fft=True,
                                                                              device=DEVICE)
        probe_weights = faceted_visualization.visualizer.main.get_probe_weights(model_location=probe_location,
                                                                                device=DEVICE)
        if current_sweep_config[constants.OPTIMIZER].casefold() == "ADAM".casefold():
            optimizer = torch.optim.Adam(params=params, lr=current_sweep_config[constants.LEARNING_RATE])
        elif current_sweep_config[constants.OPTIMIZER].casefold() == "ADAMW".casefold():
            optimizer = torch.optim.AdamW(params=params, lr=current_sweep_config[constants.LEARNING_RATE])
        elif current_sweep_config[constants.OPTIMIZER].casefold() == "SGD".casefold():
            optimizer = torch.optim.SGD(params=params, lr=current_sweep_config[constants.LEARNING_RATE])
        else:
            logger.error("Unknown optimizer type [ %s ] provided", current_sweep_config[constants.OPTIMIZER])
            raise RuntimeError(f"Unknown optimizer type [ {current_sweep_config[constants.OPTIMIZER]} ] provided")

        transforms = faceted_visualization.visualizer.image.consolidate_transforms(
            use_clip_transforms=current_sweep_config[constants.USE_TRANSFORMS],
            use_standard_transforms=current_sweep_config[constants.USE_STD_TRANSFORMS],
            clip_transforms=clip_transforms
        )

        logger.info("Final list of transforms = %s", transforms.transforms if transforms is not None else "NA")

        wandb_object = faceted_visualization.visualizer.wb.WandB(
            is_sweep=True,
            enabled=True,
            api_key=None,
            entity=None,
            run_name=None,
            project=None,
            config=None,
        )
        
        if current_sweep_config[constants.OBJECTIVE] == "neuron":
            image_name = f"{current_sweep_config[constants.VISUALIZATION_LAYER]}:{current_sweep_config[constants.CHANNEL]}_mid_neuron"
        else:
            image_name = f"{current_sweep_config[constants.VISUALIZATION_LAYER]}:{current_sweep_config[constants.CHANNEL]}"
            
        faceted_visualization.visualizer.render.optimize(
            num_iterations=current_sweep_config[constants.NUMBER_OF_ITERATIONS],
            transforms=transforms,
            image_function=image_f,
            model=model,
            channel=current_sweep_config[constants.CHANNEL],
            objective=current_sweep_config[constants.OBJECTIVE],
            layer=current_sweep_config[constants.VISUALIZATION_LAYER],
            linear_probe_layer=current_sweep_config[constants.LINEAR_PROBE_LAYER],
            probe_weights=probe_weights,
            optimizer=optimizer,
            model_hook=model_hook,
            neuron_x=current_sweep_config.get(constants.NEURON_X, None),
            neuron_y=current_sweep_config.get(constants.NEURON_Y, None),
            wandb_object=wandb_object,
            run_id=run.id,
            device=DEVICE,
            image_name=image_name
            )

        logger.info("Completed run [ %s ]", run.id)

In [16]:
sweep_id = wandb.sweep(sweep_config, project="idl-project-flowers-facet", entity="rohanprasad")
logger.info("Sweep ID = %s", sweep_id)
wandb.agent(sweep_id, function=perform_sweep)

2023-12-04 18:00:50,597 [MainThread  ] [INFO ]  Sweep ID = cuq4fdws
2023-12-04 18:00:50,601 [MainThread  ] [INFO ]  Starting sweep agent: entity=None, project=None, count=None


Create sweep with ID: cuq4fdws
Sweep URL: https://wandb.ai/rohanprasad/idl-project-flowers-facet/sweeps/cuq4fdws


wandb: Agent Starting Run: 0kbar4bx with config:
wandb: 	channel: 1
wandb: 	learning_rate: 0.05
wandb: 	linear_probe_layer: layer1_0_conv3
wandb: 	model: RN50x4
wandb: 	num_iterations: 512
wandb: 	objective: neuron
wandb: 	optimizer: AdamW
wandb: 	random_seed: 18630595
wandb: 	use_std_transforms: True
wandb: 	use_transforms: True
wandb: 	visualization_layer: layer4_5_conv3
wandb: Currently logged in as: rohanprasad. Use `wandb login --relogin` to force relogin


2023-12-04 18:01:02,611 [Thread-6 (_r] [INFO ]  Starting sweep [ run  0kbar4bx ] with config = 
{'channel': 1, 'learning_rate': 0.05, 'linear_probe_layer': 'layer1_0_conv3', 'model': 'RN50x4', 'num_iterations': 512, 'objective': 'neuron', 'optimizer': 'AdamW', 'random_seed': 18630595, 'use_std_transforms': True, 'use_transforms': True, 'visualization_layer': 'layer4_5_conv3'}
2023-12-04 18:01:02,613 [Thread-6 (_r] [INFO ]  Setting random seed to 18630595
2023-12-04 18:01:02,615 [Thread-6 (_r] [INFO ]  Loading CLIP model [ RN50x4 ].
2023-12-04 18:01:09,130 [Thread-6 (_r] [INFO ]  Finished loading model [ RN50x4 ]
2023-12-04 18:01:09,132 [Thread-6 (_r] [INFO ]  Finished loading transforms [ Compose(
    Resize(size=288, interpolation=bicubic, max_size=None, antialias=warn)
    CenterCrop(size=(288, 288))
    Normalize(mean=(0.48145466, 0.4578275, 0.40821073), std=(0.26862954, 0.26130258, 0.27577711))
) ]
2023-12-04 18:01:09,133 [Thread-6 (_r] [INFO ]  Registering hooks.
2023-12-04 18:01:

loss,█▃▂▁▁▁▁
loss,-1.51074


...

loss,██▇▇▁▃▇
loss,-0.53271


wandb: Agent Starting Run: kow8px6v with config:
wandb: 	channel: 2560
wandb: 	learning_rate: 0.05
wandb: 	linear_probe_layer: layer3_9_relu3
wandb: 	model: RN50x4
wandb: 	num_iterations: 512
wandb: 	objective: channel
wandb: 	optimizer: AdamW
wandb: 	random_seed: 18630595
wandb: 	use_std_transforms: True
wandb: 	use_transforms: True
wandb: 	visualization_layer: layer4_5_conv3


2023-12-04 19:06:57,354 [Thread-244 (] [INFO ]  Starting sweep [ run  kow8px6v ] with config = 
{'channel': 2560, 'learning_rate': 0.05, 'linear_probe_layer': 'layer3_9_relu3', 'model': 'RN50x4', 'num_iterations': 512, 'objective': 'channel', 'optimizer': 'AdamW', 'random_seed': 18630595, 'use_std_transforms': True, 'use_transforms': True, 'visualization_layer': 'layer4_5_conv3'}
2023-12-04 19:06:57,356 [Thread-244 (] [INFO ]  Setting random seed to 18630595
2023-12-04 19:06:57,357 [Thread-244 (] [INFO ]  Loading CLIP model [ RN50x4 ].
2023-12-04 19:07:02,039 [Thread-244 (] [INFO ]  Finished loading model [ RN50x4 ]
2023-12-04 19:07:02,041 [Thread-244 (] [INFO ]  Finished loading transforms [ Compose(
    Resize(size=288, interpolation=bicubic, max_size=None, antialias=warn)
    CenterCrop(size=(288, 288))
    Normalize(mean=(0.48145466, 0.4578275, 0.40821073), std=(0.26862954, 0.26130258, 0.27577711))
) ]
2023-12-04 19:07:02,042 [Thread-244 (] [INFO ]  Registering hooks.
2023-12-04 19

loss,▅▆▁█▄▂▃
loss,-1.23926


wandb: Sweep Agent: Waiting for job.
wandb: Sweep Agent: Exiting.
